In [1]:
dataset_questions = [
    "What was my hemoglobin level in the most recent report?",
    "What medications am I currently prescribed, and what are their dosages?",
    "Which hospital did I last visit for my medical visit?",
    "What was the result of my last cholesterol test?",
    "How many times have I been diagnosed with hypertension?",
    "What are the recommendations for managing my diabetes?",
    "What is the contact number of the hospital where I had my last appointment?",
    "What was the total payment for my last medical appointment?",
    "What diagnosis was given by Dr. Jane Doe during my last visit?",
    "What are the normal reference ranges for a glucose test?",
    "How often should I take my prescribed Lisinopril?",
    "What was the interpretation of my last creatinine test result?",
    "What are the tests for which I have a high interpretation?",
    "Which is recommended for my allergy diagnosis?",
    "What is the dosage of the medicine Oseltamivir?",
    "Can you provide the address of the Kindred Hospital I visited in Chicago?",
    "What was my platelet count in the most recent blood report?",
    "Who diagnosed me with flu last year?",
    "What are the dates of the appointments I have with Dr. Michael Smith?",
    "How much did I pay for my medical appointments?",
    "What is the dosage and frequency for the medicine Metformin that I am taking?",
    "Can you list all the doctors I have seen at the 4th hospital in Miami?",
    "What recommendations were given for my last diagnosis of gastritis?",
    "How many times have I been prescribed Cetirizine?",
    "What was the result of my urinalysis report from two visits ago?",
    "Can you provide a summary of my medical reports from the last six months?",
    "What are the test units for the hemoglobin A1c in my reports?",
    "How many different types of reports have I had?",
    "What was the highest payment I made for a single appointment and when?",
    "What is the latest recommendation for my hypertension management?",
    "What are the common recommendations for colds according?",
    "What was the result of my Hemoglobin A1c in the last report?",
    "What was the interpretation of my Hemoglobin A1c in the last report?",
    "What was the interpretation of my bilirubin levels in the last report?",
    "What are all the dates of all the appointments I've had with cardiologists?",
    "How many times have I visited Cedars-Sinai for appointments?",
    "What are the normal reference ranges for neutrophils?",
    "What is the email of doctor Michael Smith?",
    "What is the email of doctor Jane Doe?",
    "What is the phone number of doctor John Doe?",
    "What is the phone number of doctor Maria Garcia?",
    "What hospital does Jane Doe work at?",
    "What kind of doctor is Michael Smith?",
    "What specialization is Maria Garcia?",
    "What is the phone number of NYU Langone?",
    "What is the address of Cedars-Sinai Medical Center?",
    "How many hospitals do I have listed?",
    "What units is the Cholesterol test in?"
]

In [6]:
"""
Module Name: main.py
Description: Uses a LLM to answer a user question relating to an SQLite database
Author: Ted Sither
Date: 2024-06-09
"""


from langchain_community.utilities.sql_database import SQLDatabase
import os
from helper import generate_query, generate_response, SUBCHAIN_PROMPT, FULLCHAIN_PROMPT
from AnyScaleLLM import AnyScaleLLM






ANYSCALE_API_KEY= os.getenv("ANYSCALE_API_KEY").strip()
ANYSCALE_BASE_URL = os.getenv("ANYSCALE_BASE_URL")

MODEL = "mistralai/Mistral-7B-Instruct-v0.1"                #### MODELS FROM ANYSCALE
                                                            #  "meta-llama/Meta-Llama-3-8B-Instruct"
                                                            #  "mistralai/Mistral-7B-Instruct-v0.1"
                                                            #  "meta-llama/Meta-Llama-3-70B-Instruct"
                                                            #  "codellama/CodeLlama-70b-Instruct-hf"




llm =AnyScaleLLM(model_name="gpt-4o", api_key="AnbkO5W0aXPHWzaE8LOKABbaLtuyEEpJycOYpDNedDU=", base_url="https://api.langdock.com/openai/eu/v1") #instantiate anyscale llm object
 

DB = SQLDatabase.from_uri("sqlite:////Users/mymac/LLM/Personal-Medical-Assistant/backend/llama_index/med_assist.db")



predictions = []

for q in dataset_questions:

    try:
        query = generate_query(llm=llm, template=SUBCHAIN_PROMPT, question=q, db=DB)

        # print(f"Predicted DB Query: {query}\n\n")

        response = generate_response(llm=llm, query=query, template=FULLCHAIN_PROMPT, question=q, db=DB)

        predictions.append(response)
    
    except Exception as e:
        predictions.append("Error")



In [7]:
predictions

['Your hemoglobin level in the most recent report was 19.0.',
 'You are currently prescribed the following medications and dosages:\n\n- Paracetamol: 500 mg\n- Cetirizine: 10 mg\n- Lisinopril: 10 mg',
 'Kindred Hospital Chicago North',
 'The result of your last cholesterol test was 180.0.',
 'You have been diagnosed with hypertension 3 times.',
 'Monitor blood sugar levels and follow a diabetic diet.',
 'The contact number of the hospital where you had your last appointment is 8375621.',
 'The total payment for your last medical appointment was $70.00.',
 'The diagnosis given by Dr. Jane Doe during your last visit was "Hypertension".',
 'The normal reference ranges for a glucose test are 70-100.',
 'You should take your prescribed Lisinopril once daily.',
 'Error',
 'Hemoglobin, Cholesterol, Glucose',
 'Avoid allergens and take antihistamines.',
 'The dosage of the medicine Oseltamivir is 75.',
 'The address of the Kindred Hospital you visited in Chicago is 2544 W Montrose Ave, Chicago